In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [2]:
with sqlite3.connect("../laliga.sqlite") as con:
    df=pd.read_sql_query("SELECT * FROM Matches",con)
    dr=pd.read_sql_query("SELECT * FROM Predictions",con)

df = df.dropna(subset=['score'])

In [3]:
df["score_home_team"] = df["score"].str.split(":").str[0].astype(float)
df["score_away_team"] = df["score"].str.split(":").str[1].astype(float)
df["goal_difference"] = df["score_home_team"] - df["score_away_team"]

df["match_result"] = np.where(df['score_home_team'] > df['score_away_team'], '1', np.where(df['score_home_team'] < df['score_away_team'], '2', 'X'))

In [27]:
def func_home_wins(data):
    return (data[data == '1']).count()

def func_away_wins(data):
    return (data[data == '2']).count()

def func_tie(data):
    return (data[data == 'X']).count()

df_class_home = df.groupby(['division', 'season', 'home_team']).agg(
    GF=pd.NamedAgg(column='score_home_team', aggfunc='sum'),
    GA=pd.NamedAgg(column='score_away_team', aggfunc='sum'),
    W=pd.NamedAgg(column='match_result', aggfunc=func_home_wins),
    L=pd.NamedAgg(column='match_result', aggfunc=func_away_wins),
    T=pd.NamedAgg(column='match_result', aggfunc=func_tie)
).reset_index()

df_class_away = df.groupby(['division', 'season', 'away_team']).agg(
    GF=pd.NamedAgg(column='score_away_team', aggfunc='sum'),
    GA=pd.NamedAgg(column='score_home_team', aggfunc='sum'),
    W=pd.NamedAgg(column='match_result', aggfunc=func_away_wins),
    L=pd.NamedAgg(column='match_result', aggfunc=func_home_wins),
    T=pd.NamedAgg(column='match_result', aggfunc=func_tie)
).reset_index()

df_class_away.rename(columns={'away_team': 'team'}, inplace=True)
df_class_home.rename(columns={'home_team': 'team'}, inplace=True)

df_classification = df_class_away.merge(df_class_home, how='outer')
df_classification = df_classification.groupby(['season', 'team', 'division']).sum().reset_index()
# df_classification['GF'] = df_classification['GF'].shift(1)

df_classification['GD'] = df_classification['GF'] - df_classification['GA']
df_classification['Pts'] = (df_classification['W']) * 3 + df_classification['T']

df_classification['year_of_start'] = df_classification['season'].str.split("-").str[0].astype(int)

df_classification_ordered = df_classification.sort_values(by=['year_of_start'], ascending=False)
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division'], ascending=[False, True])
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division', 'Pts'], ascending=[False, True, False])
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division', 'Pts', 'GD'], ascending=[False, True, False, False])
df_classification_ordered = df_classification_ordered.sort_values(by=['year_of_start', 'division', 'Pts', 'GD', 'GF'], ascending=[False, True, False, False, False])

df_classification_ordered = df_classification_ordered.reset_index(drop=True)
df_classification_ordered['rank'] = df_classification_ordered.groupby(['year_of_start', 'division']).cumcount() + 1

df_classification_1_div = df_classification_ordered[df_classification_ordered['division']==1]
df_classification_2_div = df_classification_ordered[df_classification_ordered['division']==2]

lowest_rank_div1= df_classification_ordered.groupby(['season'])['rank'].max().reset_index()
# print(lowest_rank_div1)

df_class=pd.merge(lowest_rank_div1,df_classification_2_div, on=['season'],how='inner')
df_class['rank']=df_class['rank_x']+df_class['rank_y']

df_class=df_class.drop(columns={'rank_x','rank_y'})

df_bonico=df_class.merge(df_classification_1_div,how='outer')
display(df_bonico)

,season,team,division,GF,GA,W,L,T,GD,Pts,year_of_start,rank
0,1932-1933,Real Oviedo,2,58.0,26.0,12,3,3,32.0,39,1932,11
1,1932-1933,Athletic Madrid,2,40.0,35.0,10,4,4,5.0,34,1932,12
2,1932-1933,Real Unión,2,51.0,36.0,9,7,2,15.0,29,1932,13
3,1932-1933,Real Murcia,2,33.0,41.0,9,7,2,-8.0,29,1932,14
4,1932-1933,Dep. La Coruña,2,39.0,38.0,8,6,4,1.0,28,1932,15
...,...,...,...,...,...,...,...,...,...,...,...,...
2781,1928-1929,Athletic Madrid,1,43.0,41.0,8,8,2,2.0,26,1928,6
2782,1928-1929,Espanyol,1,32.0,38.0,7,7,4,-6.0,25,1928,7
2783,1928-1929,Catalunya,1,45.0,49.0,6,8,4,-4.0,22,1928,8
2784,1928-1929,Real Unión,1,40.0,42.0,5,11,2,-2.0,17,1928,9
